In [1]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 4,
   "id": "ca57f5dc",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      " Markdown report generated → D:/Disertatie/1.database_performance_analysis/results/reports/performance_report.md",
      " Poți converti la PDF cu:  pandoc -s performance_report.md -o report.pdf\n"
     ]
    }
   ],
   "source": [
    "# %% [markdown]\n",
    "# # 04 — Reporting Summary\n",
    "# \n",
    "# Un notebook „all‑in‑one” care **adună rezultatele** generate în etapele anterioare și\n",
    "# creează un fișier Markdown (`performance_report.md`) în `results/reports/` \n",
    "# se poate face conversie la PDF/HTML cu `pandoc` sau `jupyter nbconvert`.\n",
    "# \n",
    "# ## Conținutul raportului:\n",
    "# 1. Context & obiective\n",
    "# 2. Rezultate test statistice (paired t, Wilcoxon, efect + CI)\n",
    "# 3. Rezumat OLS & ANOVA\n",
    "# 4. Curbe de putere & lățime CI\n",
    "# 5. Performanță ML (RF & XGB) + grafice importanță\n",
    "# 6. Concluzii & recomandări\n",
    "# \n",
    "# ---\n",
    "# %%\n",
    "# 1️ Imports & paths\n",
    "from pathlib import Path\n",
    "import pandas as pd\n",
    "import datetime as dt\n",
    "\n",
    "\n",
    "BASE = Path(\"D:/Disertatie/1.database_performance_analysis\").resolve()\n",
    "TAB  = BASE / \"results\" / \"tables\"\n",
    "FIG  = BASE / \"results\" / \"figures\"\n",
    "REPORT_DIR = BASE / \"results\" / \"reports\"\n",
    "REPORT_DIR.mkdir(parents=True, exist_ok=True)\n",
    "\n",
    "# Helper to embed image in markdown \n",
    "def md_img(path, title=\"\", width=500):\n",
    "    rel = path.relative_to(BASE)\n",
    "    return f'![{title}]({rel.as_posix()}){{width=\"{width}px\"}}'\n",
    "\n",
    "# %%\n",
    "# 2️ Load key tables\n",
    "stat_sum   = pd.read_csv(TAB/\"statistical_summary.csv\")\n",
    "ols_vs_rlm = pd.read_csv(TAB/\"coef_ols_vs_rlm.csv\")\n",
    "anova_path = TAB/\"anova_index_shard.csv\"\n",
    "if anova_path.exists():\n",
    "    anova_tbl = pd.read_csv(anova_path)\n",
    "ml_metrics = pd.read_csv(TAB/\"ml_cv_metrics.csv\")\n",
    "\n",
    "# %%\n",
    "# 3️ Build Markdown report string\n",
    "now = dt.datetime.now().strftime(\"%Y-%m-%d %H:%M\")\n",
    "md = []\n",
    "md.append(f\"# Database Query Performance Report\\n\\n_Generated: {now}_\\n\")\n",
    "md.append(\"## 1. Obiectiv\\nAnaliza comparativă a timpului de execuție pentru interogări cu/ fără index și shard‑uri, folosind statistici clasice și modele de Machine‑Learning.\")\n",
    "\n",
    "md.append(\"## 2. Rezultate statistice clasice\\n\")\n",
    "md.append(stat_sum.round(3).to_markdown(index=False))\n",
    "md.append(\"\\n### Bootstrap Distributions\\n\")\n",
    "md.append(md_img(FIG/\"statistical\"/\"bootstrap_index.png\", \"Index bootstrap\"))\n",
    "md.append(md_img(FIG/\"statistical\"/\"bootstrap_shards.png\", \"Shards bootstrap\"))\n",
    "\n",
    "md.append(\"\\n## 3. Modele liniare & ANOVA\\n\")\n",
    "md.append(\"### Coeficienți OLS vs Robuști\\n\")\n",
    "md.append(ols_vs_rlm.round(3).to_markdown())\n",
    "if anova_path.exists():\n",
    "    md.append(\"\\n### ANOVA 2×2 (Index × Shard)\\n\")\n",
    "    md.append(anova_tbl.round(3).to_markdown())\n",
    "\n",
    "md.append(\"\\n## 4. Putere & Learning Curves\\n\")\n",
    "md.append(md_img(FIG/\"power\"/\"power_curve.png\", \"Power curve\"))\n",
    "md.append(md_img(FIG/\"power\"/\"ci_learning_curve.png\", \"CI learning curve\"))\n",
    "\n",
    "md.append(\"\\n## 5. Modele ML\\n\")\n",
    "md.append(ml_metrics.round(3).to_markdown(index=False))\n",
    "md.append(md_img(FIG/\"ml\"/\"rf_feature_importance.png\", \"RF importance\", 400))\n",
    "if (FIG/\"ml\"/\"xgb_feature_importance.png\").exists():\n",
    "    md.append(md_img(FIG/\"ml\"/\"xgb_feature_importance.png\", \"XGB importance\", 400))\n",
    "\n",
    "md.append(\"\\n## 6. Concluzii\\n* Indexarea reduce timpul mediu cu **{:+.1f} ms** și are un efect de mărime _d_ ≈ {stat_sum.loc[0,'cohen_d']:.2f}.\\n* Sharding oferă un câștig suplimentar mediu de **{stat_sum.loc[1,'mean_diff']:.1f} ms**.\\n* Modelele ML confirmă că `documents_returned` și timpul *fără* index sunt cei mai buni predictori.\\n\")\n",
    "\n",
    "report_path = REPORT_DIR/\"performance_report.md\"\n",
    "report_path.write_text(\"\\n\\n\".join(md), encoding=\"utf-8\")\n",
    "print(\" Markdown report generated →\", report_path)\n",
    "print(\" Poți converti la PDF cu:  pandoc -s performance_report.md -o report.pdf\")\n"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "venv",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.11"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}

{'cells': [{'cell_type': 'code',
   'execution_count': 4,
   'id': 'ca57f5dc',
   'metadata': {},
   'outputs': [{'name': 'stdout',
     'output_type': 'stream',
     'text': [' Markdown report generated → D:/Disertatie/1.database_performance_analysis/results/reports/performance_report.md',
      ' Poți converti la PDF cu:  pandoc -s performance_report.md -o report.pdf\n']}],
   'source': ['# %% [markdown]\n',
    '# # 04 — Reporting Summary\n',
    '# \n',
    '# Un notebook „all‑in‑one” care **adună rezultatele** generate în etapele anterioare și\n',
    '# creează un fișier Markdown (`performance_report.md`) în `results/reports/` \n',
    '# se poate face conversie la PDF/HTML cu `pandoc` sau `jupyter nbconvert`.\n',
    '# \n',
    '# ## Conținutul raportului:\n',
    '# 1. Context & obiective\n',
    '# 2. Rezultate test statistice (paired\xa0t, Wilcoxon, efect + CI)\n',
    '# 3. Rezumat OLS & ANOVA\n',
    '# 4. Curbe de putere & lățime CI\n',
    '# 5. Performanță ML (RF & XGB)